<a href="https://colab.research.google.com/github/simodepth96/technical-SEO/blob/main/JS_vs_TXT_Crawl_Comparison_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Compare JS vs TXT-based Crawls from Screaming Frog
How much JavaScript impact your target site's raw HTML?  
Use this framework to benchmark unwanted discrepancies between two crawls performed with multiple approaches.

## Use Cases
Comparing two crawls is useful when dealing with **redesigns, migrations, and activity monitoring.**

We use this Colab to spot inconsistencies between different versions of the same site (**JS vs Non-JS**, **Mobile vs Desktop**, **Googlebot vs Normal User Agent**), especially during an SEO audit.


---


##How it works
We will load two reports on a Drive folder, then we’ll access these files with Colab to manipulate them and create a new Google Spreadsheet with the difference between them.

**⚠️ Please note that if the the Comparison spreadsheet is returned as an empty file, then there are no differences between the JavaScript and the Text-based crawl**

##What changes detect
Given two crawls we are going to check:

- Newly found pages - any URL in the new crawl that isn’t in the old crawl
- Newly lost pages - any URL in the old crawl that isn’t in the new crawl
- Indexation changes - i.e. Any URL which is now canonicalized or was noindexed
- Status code changes - i.e. Any URL which was redirected but is now code 200
- URL-level Canonical Tag changes
- URL-level Title Tag or Meta Description changes
- URL-level H1 or H2 changes



First of all, we need to install `pygsheets` , a Python packet that we'll use to create or edit Spreadsheet in our Google account.

In [ ]:
!pip install --upgrade -q pygsheets

     |████████████████████████████████| 147 kB 13.0 MB/s 


Colab integrates with Google Docs and Drive ecosystem, giving you a significant boost when analyzing data or testing out new things quickly.

Our `internal_all` reports are uploaded on Google Drive, so we'll need to connect to Google Drive.

 We only need to get our Authorization Code. 

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In this case our reports are in the folder /diffsf/.
![Drive](https://i.gyazo.com/70cce25d55340f1977272d1e7a94825f.png)

In [ ]:
root_path = '/content/drive/My Drive/Crawls/'

Colab (and Python Notebooks in general) gives you the possibility to use Unix command, adding a ! before the command. You can explore your Drive folder through the Unix command `ls` .

In [ ]:
!ls -lah drive/My\ Drive/Crawls

total 35K
-rw------- 1 root root 15K Oct 23 15:16 internal_all.xlsx
-rw------- 1 root root 21K Oct 23 15:25 JS_internal_all.xlsx


Numpy and Pandas are useful packets for data manipulation. They are so powerful and can manage huge files easily. 

In [ ]:
import numpy as np
import pandas as pd

Now we need to connect to Spreadsheets. To do this, we need an auth key for accessing Spreadsheet API v4.

In [ ]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
sh_name = input("Type the filename of your sheets...")

Type the filename of your sheets...Test SF Crawls Comparison


Now we'll create a spreadsheet with the name we have given via `sh_name` .

In [ ]:
sh = gc.create(sh_name)

## Open two crawls to compare

Replace `internal_all_21102019.csv` and `internal_all_22102019.csv` with your own files.

In [ ]:
internal_all_JS = pd.read_excel(root_path+'JS_internal_all.xlsx',header=0)

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
internal_all_TXT = pd.read_excel(root_path+'internal_all.xlsx',header=0)

Here we'll filter out all non-html requests.

In [ ]:
JS_crawl = internal_all_JS[internal_all_JS['Content Type'].str.contains("text/html", na=False)]

In [ ]:
TXT_crawl = internal_all_TXT[internal_all_TXT['Content Type'].str.contains("text/html", na=False)]

[Pandas info method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) is useful when we need to know quickly what reports contains.

In [ ]:
JS_crawl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 61
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Address                         32 non-null     object 
 1   Content Type                    32 non-null     object 
 2   Status Code                     32 non-null     float64
 3   Status                          9 non-null      object 
 4   Indexability                    32 non-null     object 
 5   Indexability Status             9 non-null      object 
 6   Title 1                         23 non-null     object 
 7   Title 1 Length                  32 non-null     float64
 8   Title 1 Pixel Width             32 non-null     float64
 9   Meta Description 1              23 non-null     object 
 10  Meta Description 1 Length       32 non-null     float64
 11  Meta Description 1 Pixel Width  32 non-null     float64
 12  Meta Keywords 1                 0 non-

In [ ]:
TXT_crawl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 32
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Address                         32 non-null     object 
 1   Content Type                    32 non-null     object 
 2   Status Code                     32 non-null     float64
 3   Status                          32 non-null     object 
 4   Indexability                    32 non-null     object 
 5   Indexability Status             9 non-null      object 
 6   Title 1                         23 non-null     object 
 7   Title 1 Length                  32 non-null     float64
 8   Title 1 Pixel Width             32 non-null     float64
 9   Meta Description 1              23 non-null     object 
 10  Meta Description 1 Length       32 non-null     float64
 11  Meta Description 1 Pixel Width  32 non-null     float64
 12  Meta Keywords 1                 0 non-

## New found Pages



Through `pd.merge` we are going to merge the JavaScript and the Text-based crawl using the URLs as their common key.

In [ ]:
_new_found_pages = pd.merge(JS_crawl,\
                            TXT_crawl,\
                            suffixes=('_JS_crawl', '_TXT_crawl'),\
                            on='Address',\
                            how='outer')

We'll filter out all null values.

In [ ]:
_new_found_pages = _new_found_pages[_new_found_pages['Status Code_JS_crawl'].isna()]

And keep only the columns that contains `_current` (from the latest crawl) and `Address`. 

In [ ]:
new_found_pages = _new_found_pages.filter(regex='Address|\_TXT_crawl') 

In [ ]:
new_found_pages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 58 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Address                                   0 non-null      object 
 1   URL Encoded Address_JS_crawl              0 non-null      object 
 2   Content Type_TXT_crawl                    0 non-null      object 
 3   Status Code_TXT_crawl                     0 non-null      float64
 4   Status_TXT_crawl                          0 non-null      object 
 5   Indexability_TXT_crawl                    0 non-null      object 
 6   Indexability Status_TXT_crawl             0 non-null      object 
 7   Title 1_TXT_crawl                         0 non-null      object 
 8   Title 1 Length_TXT_crawl                  0 non-null      float64
 9   Title 1 Pixel Width_TXT_crawl             0 non-null      float64
 10  Meta Description 1_TXT_crawl              0 non-nu

Now we're going to evaluate if the `new_found_pages` DataFrame is empty. If not, we are going to create a new worksheet and append the DataFrame.

In [ ]:
if len(new_found_pages['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now we need to select the first Sheet in our Spreadsheet. 
  #To access it, we need to select it through an index (0-based).
  wks = sh[0]
  #set_dataframe is a method that injects a DataFrame into a spreadsheet. 
  #The first argument is the DF and the second arg is the cell where we start filling.
  wks.set_dataframe(new_found_pages, 'A1',fit=True)
  #We can also change the title of our worksheet, in this way
  sh.sheet1.title="New Found Pages"

No changes detected!


## Newly lost pages

In [ ]:
_new_lost_pages = _new_found_pages[_new_found_pages['Status Code_TXT_crawl'].isna()]

In [ ]:
new_lost_pages = _new_found_pages.filter(regex='Address|\_JS_crawl').dropna()

In [ ]:
if len(new_lost_pages['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('New Lost Pages')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(new_lost_pages, 'A1',fit=True)

No changes detected!


## Changed Status Code

In [ ]:
_changed_status_code = pd.merge(previous[['Address','Status Code']],\
                                current[['Address','Status Code']],\
                                suffixes=('_prev', '_current'),\
                                on='Address',\
                                how='inner')

In [ ]:
_changed_status_code['diff'] = np.where(_changed_status_code['Status Code_prev'] == _changed_status_code['Status Code_current'], 'no change', 'changed')

In [ ]:
changed_status_code = _changed_status_code[_changed_status_code['diff'] == 'changed']

In [ ]:
if len(changed_status_code['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Status Code')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_status_code, 'A1',fit=True)

## changed indexation

In [ ]:
_changed_indexation = pd.merge(JS_crawl[['Address','Status Code', 'Indexability','Indexability Status']],\
                                TXT_crawl[['Address','Status Code', 'Indexability','Indexability Status']],\
                                suffixes=('_JS_crawl', '_TXT_crawl'), on='Address', how='inner')

In [ ]:
_changed_indexation['diff'] = np.where(_changed_indexation['Indexability_JS_crawl'] == _changed_indexation['Indexability_TXT_crawl'], 'no change', 'changed')

In [ ]:
changed_indexation = _changed_indexation[['Address','Indexability_JS_crawl','Indexability Status_JS_crawl','Indexability_TXT_crawl','Indexability Status_TXT_crawl','diff']]

In [ ]:
changed_indexation = changed_indexation[changed_indexation['diff'] == 'changed']

In [ ]:
if len(changed_indexation['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Indexation')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_indexation, 'A1',fit=True)

No changes detected!


## changed meta


In [ ]:
_changed_meta = pd.merge(JS_crawl[['Address','Title 1', 'Meta Description 1']],\
                                TXT_crawl[['Address','Title 1', 'Meta Description 1']],\
                                suffixes=('_JS_crawl', '_TXT_crawl'), on='Address', how='inner')

In [ ]:
_changed_meta['diff_title'] = np.where(_changed_meta['Title 1_JS_crawl'] == _changed_meta['Title 1_TXT_crawl'], 'no change', 'changed')

In [ ]:
_changed_meta['diff_desc'] = np.where(_changed_meta['Meta Description 1_JS_crawl'] == _changed_meta['Meta Description 1_TXT_crawl'], 'no change', 'changed')

In [ ]:
changed_title = _changed_meta[_changed_meta['diff_title'] == 'changed'].dropna().filter(regex='Address|^Title.+|diff\_title')

In [ ]:
changed_desc= _changed_meta[_changed_meta['diff_desc'] == 'changed'].dropna().filter(regex='Address|.+Description.+|diff\_desc')

In [ ]:
if len(changed_title['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Title')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_title, 'A1',fit=True)

No changes detected!


In [ ]:
if len(changed_desc['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Description')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_desc, 'A1',fit=True)

No changes detected!


## changed H1 tags

In [ ]:
import re

In [ ]:
changed_h1 = pd.merge(JS_crawl.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3),\
                            TXT_crawl.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3),\
                            suffixes=('_JS_crawl', '_TXT_crawl'),\
                            on='Address',\
                            how='inner')

In [ ]:
changed_h1.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
changed_h1_cols = TXT_crawl.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3).columns.tolist()

In [ ]:
r = re.compile("^H1\-\d{1,2}")

In [ ]:
newlist = list(filter(r.match, changed_h1_cols))

In [ ]:
for i in range(1,len(newlist)+1):
    new_diff = "H1-"+str(i)
    new = f"{new_diff}_TXT_crawl"
    old = f"{new_diff}_JS_crawl"
    _tmp = f"diff-{new_diff}"
    changed_h1[_tmp] = changed_h1[new] != changed_h1[old]

In [ ]:
changed_h1.columns =[column.replace("-", "_") for column in changed_h1.columns] 

In [ ]:
diff_col_lst = changed_h1.filter(regex='diff\_H1').columns.tolist()

In [ ]:
exp = ' or '.join(diff_col_lst)

In [ ]:
changed_h1 = changed_h1.query(exp)

In [ ]:
if len(changed_h1['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed H1')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_h1, 'A1',fit=True)

No changes detected!


## Changed H2

In [ ]:
changed_h2 = pd.merge(JS_crawl.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3),\
                TXT_crawl.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3),\
                suffixes=('_JS_crawl', '_TXT_crawl'),\
                on='Address',\
                how='inner')

In [ ]:
changed_h2.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
changed_h2_cols = TXT_crawl.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3).columns.tolist()

In [ ]:
r = re.compile("^H2\-\d{1,2}")

In [ ]:
newlist = list(filter(r.match, changed_h2_cols))

In [ ]:
for i in range(1,len(newlist)+1):
    new_diff = "H2-"+str(i)
    new = f"{new_diff}_TXT_crawl"
    old = f"{new_diff}_JS_crawl"
    _tmp = f"diff-{new_diff}"
    changed_h2[_tmp] = changed_h2[new] != changed_h2[old]

In [ ]:
changed_h2.columns =[column.replace("-", "_") for column in changed_h2.columns] 

In [ ]:
diff_col_lst = changed_h2.filter(regex='diff\_H2').columns.tolist()

In [ ]:
exp = ' or '.join(diff_col_lst)

In [ ]:
changed_h2 = changed_h2.query(exp)

In [ ]:
if len(changed_h2['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed H2')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_h2, 'A1',fit=True)

## Changed canonicals

In [ ]:
_changed_canonical = pd.merge(JS_crawl[['Address','Status Code','Canonical Link Element 1']],TXT_crawl[['Address','Status Code','Canonical Link Element 1']],\
                               suffixes=('_JS_crawl', '_TXT_crawl'), on='Address', how='inner')

In [ ]:
_changed_canonical.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
_changed_canonical['diff_canonical'] = np.where(_changed_canonical['Canonical Link Element 1_JS_crawl'] == _changed_canonical['Canonical Link Element 1_TXT_crawl'], 'no change', 'changed')

In [ ]:
changed_canonical = _changed_canonical[_changed_canonical['diff_canonical'] == 'changed']

In [ ]:
if len(changed_canonical['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Canonicals')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_canonical, 'A1',fit=True)

No changes detected!
